In [56]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [57]:
cities_csv = "cities1.csv"
cities_df = pd.read_csv(cities_csv)

cities_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,barrow,71.2906,-156.7887,-5.78,77,0,11.50,US,1649926096
1,1,hithadhoo,-0.6000,73.0833,83.73,73,100,10.04,MV,1649926096
2,2,vao,-22.6667,167.4833,77.50,70,57,11.99,NC,1649926096
3,3,rikitea,-23.1203,-134.9692,78.13,70,4,17.60,PF,1649926096
4,4,popondetta,-8.7537,148.2534,76.44,90,96,2.77,PG,1649926096


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [58]:
gmaps.configure(api_key = g_key)

In [59]:
cities_df.info()

locations = cities_df[["Lat", "Lng"]].astype(float)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551 entries, 0 to 550
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  551 non-null    int64  
 1   City        551 non-null    object 
 2   Lat         551 non-null    float64
 3   Lng         551 non-null    float64
 4   Max Temp    551 non-null    float64
 5   Humidity    551 non-null    int64  
 6   Cloudiness  551 non-null    int64  
 7   Wind Speed  551 non-null    float64
 8   Country     549 non-null    object 
 9   Date        551 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 43.2+ KB


In [102]:
cities_df["Humidity"].astype(float)

0      77.0
1      73.0
2      70.0
3      70.0
4      90.0
       ... 
546    86.0
547    64.0
548    90.0
549    86.0
550    73.0
Name: Humidity, Length: 551, dtype: float64

In [107]:
# Create humidity heatmap
humidity = cities_df["Humidity"]

locations = cities_df[["Lat", "Lng"]].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=5,
                                point_radius=5)

fig.add_layer(heat_layer)

In [108]:
%matplotlib inline

In [109]:
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [110]:
cities_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,barrow,71.2906,-156.7887,-5.78,77,0,11.50,US,1649926096
1,1,hithadhoo,-0.6000,73.0833,83.73,73,100,10.04,MV,1649926096
2,2,vao,-22.6667,167.4833,77.50,70,57,11.99,NC,1649926096
3,3,rikitea,-23.1203,-134.9692,78.13,70,4,17.60,PF,1649926096
4,4,popondetta,-8.7537,148.2534,76.44,90,96,2.77,PG,1649926096


In [111]:
type(cities_df["Max Temp"])

pandas.core.series.Series

In [112]:
(cities_df["Max Temp"]).astype(float)

0      -5.78
1      83.73
2      77.50
3      78.13
4      76.44
       ...  
546    65.75
547    93.09
548    45.95
549    39.02
550    17.85
Name: Max Temp, Length: 551, dtype: float64

In [113]:
(cities_df["Cloudiness"]).astype(int)

0        0
1      100
2       57
3        4
4       96
      ... 
546     66
547     34
548    100
549    100
550     40
Name: Cloudiness, Length: 551, dtype: int32

In [67]:
(cities_df["Wind Speed"]).astype(float)

0      11.50
1      10.04
2      11.99
3      17.60
4       2.77
       ...  
546     6.87
547     4.18
548     5.37
549    13.51
550    13.80
Name: Wind Speed, Length: 551, dtype: float64

In [114]:
# Ideal weather coniditions 
cities_df

ideal_weather = (cities_df["Max Temp"] >= 80) & (cities_df["Cloudiness"] < 10) & (cities_df["Wind Speed"] < 10)

#ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = cities_df.loc[ideal_weather, :]
ideal_weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
48,48,victoria,22.2855,114.1577,88.18,64,1,8.01,HK,1649925962
129,129,zaranj,30.9667,61.8833,100.36,5,1,1.97,AF,1649926116
335,335,orzu,37.5593,68.8184,86.40,21,0,3.24,TJ,1649926143
463,463,isangel,-19.5500,169.2667,80.10,81,5,5.84,VU,1649926163


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [115]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# New hotel dataframe to add Hotel Name to
hotel_df = ideal_weather_df

# Find out hotels in above cities by using API
params = {"radius": 5000,
         "type": "lodging",
         "key": g_key}

# Loop through hotel_df to find out hotel for each long and lat
for index, row in hotel_df.iterrows():
    # Get lat and long 
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Assemble URL and make API request 
    print(f"Retrieving results for index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    results = response["results"]
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving results for index 48: victoria.
Closest hotel is Mini Hotel Central.
------------
Retrieving results for index 129: zaranj.


C:\Users\shann\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\shann\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Closest hotel is کارگاه بلوک زنی سرهنگ.
------------
Retrieving results for index 335: orzu.
Missing field/result... skipping.
------------
Retrieving results for index 463: isangel.
Closest hotel is Tanna Lodge.
------------


In [117]:
# See new hotel df in ideal places

hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
48,48,victoria,22.2855,114.1577,88.18,64,1,8.01,HK,1649925962,Mini Hotel Central
129,129,zaranj,30.9667,61.8833,100.36,5,1,1.97,AF,1649926116,کارگاه بلوک زنی سرهنگ
335,335,orzu,37.5593,68.8184,86.40,21,0,3.24,TJ,1649926143,NaN
463,463,isangel,-19.5500,169.2667,80.10,81,5,5.84,VU,1649926163,Tanna Lodge


In [87]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [118]:
# Add marker layer ontop of heat map
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)
hotel_layer = gmaps.symbol_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))